In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('/Users/sebas/projects/metabolomics')
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [3]:
from src.transformers.sklearn_model import SklearnModel
import gensim
from src.load_data import LoadData
from src.config import Config
from tqdm import tqdm
from src.loader_saver import LoaderSaver
import itertools
import numpy as np
from scipy.stats import spearmanr
import dill
from src.plotting import Plotting
from src.load_data import LoadData
from src.molecule_pairs_opt import MoleculePairsOpt
from sklearn.model_selection import train_test_split
from src.train_utils import TrainUtils
from src.preprocessor import Preprocessor
import pickle
import sys
from src.config import Config
from src.parser import Parser
from datetime import datetime
from src.loader_saver import LoaderSaver
from src.molecular_pairs_set import MolecularPairsSet
from scipy.stats import spearmanr
from src.transformers.embedder import Embedder
from src.transformers.encoder import Encoder
import matplotlib.pyplot as plt
from src.transformers.CustomDatasetEncoder import CustomDatasetEncoder
from src.transformers.load_data_encoder import LoadDataEncoder
from src.transformers.load_data_unique import LoadDataUnique
from torch.utils.data import DataLoader
from src.analog_discovery.cosine_similarity import CosineSimilarity
from rdkit import Chem
from src.analog_discovery.analog_discovery import AnalogDiscovery
from src.analog_discovery.mces import MCES
from src.analog_discovery.ad_multitask import ADMultitask
import copy

## params

In [4]:
MULTITASKING=True

In [5]:
data_folder= '/Users/sebas/projects/data/'
janssen_path = data_folder + 'drug_plus.mgf'
output_janssen_file= data_folder + 'all_spectrums_janssen.pkl'
dataset_path= data_folder +'merged_gnps_nist_20240311_unique_smiles_1_million.pkl'
#model_path = data_folder + 'best_model_multitasking_min_peaks.ckpt'
#model_path = data_folder + 'best_model_multitasking_pretraining_2.ckpt'
model_path = data_folder + 'best_model_mces20_ed_20241118_precursor_randomized.ckpt'
casmi_file= '/Users/sebas/projects/data/processed_massformer/spec_df.pkl' 

In [6]:
config=Config()

In [7]:
config.D_MODEL=512
config.N_LAYERS=5


In [8]:
USE_CASMI=True

In [9]:
#similarity_name= 'simba_EDIT_DISTANCE_WITH_CONFIDENCE'
#similarity_name= 'simba_EDIT_DISTANCE'
similarity_name= 'simba_EDIT_DISTANCE_PRETRAINING'
#similarity_name= 'simba_EDIT_DISTANCE_PRETRAINING_WITHOUT_CONFIDENCE'

In [10]:
USE_MS2DEEPSCORE =False
if USE_MS2DEEPSCORE:
    similarity_name= 'simba_SIMBA_ENSEMBLED_MS2DEEPSCORE_SPEC2VEC'


In [11]:
data_folder= '/Users/sebas/projects/data/'
model_ms2d_file = data_folder + 'ms2deepscore_positive_10k_1000_1000_1000_500.hdf5'
model_spec2vec_file = data_folder + 'spec2vec_AllPositive_ratio05_filtered_201101_iter_15.model'


In [12]:
from ms2deepscore import MS2DeepScore
from ms2deepscore.models import load_model
import tensorflow as tf
from matchms import calculate_scores
from src.matchms_utils import MatchmsUtils
from spec2vec import Spec2Vec

class Ms2wrapper:
    def __init__(self, model_ms2d_file):
        self.model_ms2d = load_model(model_ms2d_file)
        self.similarity_model= MS2DeepScore(self.model_ms2d)
        
    def get_predictions_ms2deepscore(self,s1, s2):
        results_ms2_all=[]
        s_query= MatchmsUtils.from_su_to_matchms(s1)
        s_ref= MatchmsUtils.from_su_to_matchms(s2)
        with tf.device('/device:CPU:0'): #execute on cpu
                results_ms2 = calculate_scores([s_query], [s_ref], self.similarity_model)
        return results_ms2.to_array()

    def compute_similarity_matrix(self,specs_su_1, specs_su_2):
        results_ms2_all=[]
        s_query= [MatchmsUtils.from_su_to_matchms(s1) for s1 in specs_su_1]
        s_ref=  [MatchmsUtils.from_su_to_matchms(s2) for s2 in specs_su_2]
        with tf.device('/device:CPU:0'): #execute on cpu
                results_ms2 = calculate_scores(s_query, s_ref, self.similarity_model)
        return results_ms2.to_array()

class Spec2vec_wrapper:
    def __init__(self, model_ms2d_file):
        self.model = gensim.models.Word2Vec.load(model_spec2vec_file)
        self.similarity_model= Spec2Vec(
                model=self.model, intensity_weighting_power=0.5, allowed_missing_percentage=100.0
            )
    def compute_similarity_matrix(self,specs_su_1, specs_su_2):
        s_query= [MatchmsUtils.from_su_to_matchms(s1) for s1 in specs_su_1]
        s_ref=  [MatchmsUtils.from_su_to_matchms(s2) for s2 in specs_su_2]
        with tf.device('/device:CPU:0'): #execute on cpu
                results_spec2vec_all = calculate_scores(s_query, s_ref, self.similarity_model)
        return results_spec2vec_all.to_array()

In [13]:
#ms2_object=Spec2vec_wrapper(model_spec2vec_file)
ms2_object=Ms2wrapper(model_ms2d_file)

2024-11-18 10:07:29.605955: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-11-18 10:07:29.605973: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-11-18 10:07:29.605977: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-11-18 10:07:29.606018: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-18 10:07:29.606044: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## open janssen data

In [14]:
if USE_CASMI:
    all_spectrums_janssen_original=LoadData.get_all_spectrums_casmi(
        casmi_file,
        config=config,
    )
else:
    loader_saver = LoaderSaver(
            block_size=100,
            pickle_nist_path='',
            pickle_gnps_path='',
            pickle_janssen_path=output_janssen_file,
        )
    all_spectrums_janssen_original = loader_saver.get_all_spectrums(
                janssen_path,
                100000000,
                use_tqdm=True,
                use_nist=False,
                config=config,
                use_janssen=True,
            )

In [15]:
all_spectrums_janssen_processed= [copy.deepcopy(s) for s in all_spectrums_janssen_original]


In [16]:
pp=Preprocessor()
### remove extra peaks in janssen
all_spectrums_janssen_processed = [pp.preprocess_spectrum(
            s,
            fragment_tol_mass=10,
            fragment_tol_mode="ppm",
            min_intensity=0.01,
            max_num_peaks=100,
            scale_intensity=None,
        ) for s in all_spectrums_janssen_processed]

all_spectrums_janssen = [s_original for s_original, s_processed in zip(all_spectrums_janssen_original,all_spectrums_janssen_processed) if len(s_processed.mz)>=6]

## open ref data

In [17]:
with open(dataset_path, 'rb') as file:
            dataset = dill.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sebas/projects/data/merged_gnps_nist_20240311_unique_smiles_1_million.pkl'

In [ ]:
dataset['molecule_pairs_test']

In [ ]:
all_spectrums_reference_original= dataset['molecule_pairs_train'].spectrums_original +\
                    dataset['molecule_pairs_val'].spectrums_original + \
                        dataset['molecule_pairs_test'].spectrums_original


In [ ]:
import copy
all_spectrums_reference_processed= [copy.deepcopy(s) for s in all_spectrums_reference_original]

In [ ]:
all_spectrums_reference_processed = [pp.preprocess_spectrum(
            s,
            fragment_tol_mass=10,
            fragment_tol_mode="ppm",
            min_intensity=0.01,
            max_num_peaks=100,
            scale_intensity=None,
        ) for s in all_spectrums_reference_processed]
    
all_spectrums_reference = [s_original for s_original, s_processed in zip(all_spectrums_reference_original,all_spectrums_reference_processed) if len(s_processed.mz)>=6]

## Find those instances that are not in reference

In [ ]:
canon_smiles_reference = [Chem.CanonSmiles(s.smiles) for s in all_spectrums_reference]
canon_smiles_janssen =   [Chem.CanonSmiles(s.smiles) for s in all_spectrums_janssen]
janssen_indexes_in_ref= [i for i,s in enumerate(canon_smiles_janssen) if s in canon_smiles_reference]
janssen_indexes_not_in_ref = [i for i,s in enumerate(canon_smiles_janssen) if s not in canon_smiles_reference]

In [ ]:
len(janssen_indexes_in_ref),len(janssen_indexes_not_in_ref)

In [ ]:
all_spectrums_janssen = [all_spectrums_janssen[index] for index in janssen_indexes_not_in_ref]

## load encoder

In [ ]:
encoder= Encoder(model_path, D_MODEL=int(config.D_MODEL),N_LAYERS=int(config.N_LAYERS), multitasking=MULTITASKING, config=config)

## data loader

In [ ]:
dataset_janssen= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_janssen)
dataloader_janssen= DataLoader(dataset_janssen, batch_size=config.BATCH_SIZE,  num_workers=0)

In [ ]:
dataset_reference= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_reference)
dataloader_reference= DataLoader(dataset_reference, batch_size=config.BATCH_SIZE,  num_workers=0)

In [ ]:
#dataset_nist= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_nist)
#dataloader_nist= DataLoader(dataset_nist, batch_size=config.BATCH_SIZE,  num_workers=0)

In [ ]:
#dataset_gnps= LoadDataEncoder.from_spectrums_to_dataset(all_spectrums_gnps)
#dataloader_gnps= DataLoader(dataset_gnps, batch_size=config.BATCH_SIZE,  num_workers=0)

In [ ]:
for batch in dataloader_reference:
    print(batch['intensity'])
    plt.plot(batch['intensity'][10])
    break

## forward

In [ ]:
embeddings_janssen= encoder.get_embeddings(dataloader_janssen)

In [ ]:
embeddings_reference = encoder.get_embeddings(dataloader_reference)

In [ ]:
#embeddings_nist = encoder.get_embeddings(dataloader_nist)

In [ ]:
#embeddings_gnps = encoder.get_embeddings(dataloader_gnps)

## compute similarities simba

In [ ]:
similarities= CosineSimilarity.batch_cosine_similarity_combinations(embeddings_janssen, embeddings_reference)

In [ ]:
similarities.shape

## compute similarities ms2deepscore

In [ ]:
import copy
from src.spectrum_ext import SpectrumExt
from matchms.Spectrum import Spectrum
all_spectrums_reference_preprocessed=[]
for s in all_spectrums_reference:
    if s.mz[(s.mz >= 10) & (s.mz <= 1000)].shape[0]>0:
        all_spectrums_reference_preprocessed.append(s)
    else:
        all_spectrums_reference_preprocessed.append(all_spectrums_reference[0])

In [ ]:
len(all_spectrums_reference_preprocessed), len(all_spectrums_reference)

In [ ]:
if USE_MS2DEEPSCORE:
        path_file='./similarities_ms2.npy'
        if (os.path.exists(path_file)):
            similarities_ms2=np.load(path_file)
        else:
           similarities_ms2 = ms2_object.compute_similarity_matrix(all_spectrums_janssen, all_spectrums_reference_preprocessed)

## heat map

In [ ]:
plt.figure(figsize=(10,100))
plt.imshow(similarities[:, 0:10000], cmap='hot', interpolation='nearest')
#plt.colorbar()  # Add colorbar to show scale
plt.title('Heatmap of Matrix')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

## Show PCA

In [ ]:
from src.analog_discovery.pca_embeddings import PCAEmbeddings

In [ ]:
pca = PCAEmbeddings()

In [ ]:
pca.fit(embeddings_reference)

In [ ]:
pca_ref= pca.transform(embeddings_reference)

In [ ]:
pca_janssen= pca.transform(embeddings_janssen)

In [ ]:
#pca_nist= pca.transform(embeddings_nist)

In [ ]:
#pca_gnps= pca.transform(embeddings_gnps)

In [ ]:
from src.plotting import Plotting

In [ ]:
Plotting.plot_n_pca([pca_ref, pca_janssen,], ['nist+gnps','janssen', ], alpha=0.2)

In [ ]:
similarities.shape

In [ ]:
from src.tanimoto import Tanimoto
from tqdm import tqdm

In [ ]:
fp_j_all = [Tanimoto.compute_fingerprint(s.params['smiles']) for s in all_spectrums_janssen]


In [ ]:
fp_r_all = [Tanimoto.compute_fingerprint(s.params['smiles']) for s in all_spectrums_reference]

In [ ]:
N=10000
random_j= np.random.randint(0,similarities.shape[0],N)
random_r=np.random.randint(0,similarities.shape[1],N)

In [ ]:
tanimotos=np.zeros((similarities.shape[0], similarities.shape[1]))
for index_j in tqdm(random_j):
    for index_r in random_r:
        tanimotos[index_j, index_r]=Tanimoto.compute_tanimoto(fp_j_all[index_j], fp_r_all[index_r])

In [ ]:
if USE_MS2DEEPSCORE:
    plt.scatter(tanimotos[random_j, random_r].reshape(-1,1), similarities_ms2[random_j, random_r].reshape(-1,1), alpha=0.2)

In [ ]:
plt.scatter(tanimotos[random_j, random_r].reshape(-1,1), similarities[random_j, random_r].reshape(-1,1), alpha=0.2)

In [ ]:
from scipy.stats import spearmanr
x=tanimotos[random_j, random_r].reshape(-1)

y=similarities[random_j, random_r].reshape(-1)

spearmanr(x,y)

In [ ]:
_=plt.hist(similarities.reshape(-1)[0:10000000],bins=50)

## Compute new similarities based on similarities of ms2deepscore and simba

In [ ]:
USE_MS2DEEPSCORE

In [ ]:
similarities_simba=similarities.copy()

In [ ]:
# difference between similarities
#diff_sim = np.abs(similarities_simba-similarities_ms2)

In [ ]:
#mask_matrix = (similarities_ms2 > 0.5).astype(int)

In [ ]:
#mask_matrix

In [ ]:
if USE_MS2DEEPSCORE:
    #similarities = similarities_simba.copy()
    similarities= (similarities_simba + similarities_ms2)/2
    similarities= similarities*mask_matrix
    #similarities = np.minimum(similarities_simba, similarities_ms2)

In [ ]:
#similarities_simba[0,0],similarities_ms2[0,0],similarities[0,0]

## Based on the similarities compute the similarity score of the match spectrum

In [ ]:
spectrums_k_retrieved, tanimoto_k_retrieved, sim_k_retrieved, arg_max_k10 =\
                AnalogDiscovery.get_k_candidates(similarities, all_spectrums_reference, 
                                                 all_spectrums_janssen,k=10, return_arg_max_k10=True)

In [ ]:
if USE_MS2DEEPSCORE:
    similarities_simba_k_retrieved=  np.take_along_axis(similarities_simba, arg_max_k10, axis=1)
    similarities_ms2_k_retrieved=  np.take_along_axis(similarities_ms2, arg_max_k10, axis=1)
    #index=-1
    #for sim_total, simba, ms2, tani in zip(sim_k_retrieved[index], similarities_simba_k_retrieved[index], similarities_ms2_k_retrieved[index], tanimoto_k_retrieved[index]):
    #    print(f'S:{simba},MS2: {ms2},TOTAL: {sim_total} TANI:{tani}\n')

In [ ]:
## For the k spectrums retrieved, calculate the edit distance

In [ ]:
ad_multitask= ADMultitask(model_path, config=config)

In [ ]:
edit_distance_k= ad_multitask.get_edit_distance_all(all_spectrums_janssen, spectrums_k_retrieved )

In [ ]:
edit_distance_k[24]

In [ ]:
# get the confidence of the first prediction
confidence_k= [[ed[0] for ed in ed_group]for ed_group in edit_distance_k]

In [ ]:
# based on the difference between m0 and m1
#confidence_k = [[abs(m-s)/max(m,s) for s,m in zip(simba_group, ms2_group)]for simba_group, ms2_group in zip(similarities_simba_k_retrieved, similarities_ms2_k_retrieved)]

In [ ]:
all_low = np.concatenate(confidence_k)
_=plt.hist(all_low, bins=30)

In [ ]:
#ms2_deepscore_k= [[ms2_object.get_predictions_ms2deepscore(s,s_query) for s in s_group_k] for s_group_k, s_query in zip(spectrums_k_retrieved, all_spectrums_janssen)]

In [ ]:
## metric based on ms2deepscore
#new_metric_not_collapsed= [[ (sim, ms2, tani, spec.params['smiles']) \
#               for sim, ms2, tani, spec in zip(sim_group, ms2_group, tani_group, spec_group )] \
#               for sim_group, ms2_group, tani_group, spec_group  in zip(sim_k_retrieved, ms2_deepscore_k, tanimoto_k_retrieved, spectrums_k_retrieved) \
#                ]

In [ ]:
#new_metric = [[ float((m[0])/2) for m in m_group] for m_group in new_metric_not_collapsed]
#new_metric = [[ float((m[0]+m[1])/2)  for m in m_group] for m_group in new_metric_not_collapsed]


In [ ]:
# create a new metric based on tanimoto and edit distance
#categories= np.array([0,1,2,3,4,5])/5
#new_metric= [ [ (sim +  (1-edit_distance[0]) + (np.argmax(edit_distance))/5  )/3 \
#new_metric= [[ ( sim )/2 \
#new_metric= [[ ( sim + (np.argmax(edit_distance))/5)/2 \
#new_metric= [[ ( sim + np.sum(categories*edit_distance))/2 \
#               for sim, edit_distance in zip(sim_group, edit_distance_group)] \
#               for sim_group, edit_distance_group in zip(sim_k_retrieved, edit_distance_k) \
#                ]
#
#new_metric= [ [ (sim + np.sum(categories*edit_distance))/2 for sim, edit_distance in zip(sim_group, edit_distance_group)] \
#                for sim_group, edit_distance_group in zip(sim_k_retrieved, edit_distance_k)]
#new_metric

In [ ]:
good_k_indexes= [[i for i,c in enumerate(c_group) if c<1] for c_group in confidence_k ]
#good_k_indexes= [[i for i,c in enumerate(c_group) if c<0.001] for c_group in confidence_k ]
#good_k_indexes = [[i for i,c in enumerate(c_group) if c<1] for c_group in confidence_k ]
#good_k_indexes= [list(np.flip(np.argsort(m_group))) for m_group in new_metric]


# if there are no good indexes no filter
#good_k_indexes= [index_group if len(index_group)>0 else list(range(0,10)) for index_group in good_k_indexes]


In [ ]:
filt_spectrums_k_retrieved = [[ spectrums_k_retrieved[k][ind]  for ind in index_group]  for k,index_group in enumerate(good_k_indexes)]
filt_tanimoto_k_retrieved = [[  tanimoto_k_retrieved[k][ind] for ind in index_group]  for k,index_group in enumerate(good_k_indexes)]
filt_sim_k_retrieved = [[ sim_k_retrieved[k][ind] for ind in index_group]  for k,index_group in enumerate(good_k_indexes)]
filt_edit_distance_k_retrieved = [[ edit_distance_k[k][ind] for ind in index_group]  for k,index_group in enumerate(good_k_indexes)]

In [ ]:
best_indexes = [np.argmax(tani_group[0:10]) if len(tani_group)>=1 else None for tani_group in filt_tanimoto_k_retrieved ]

In [ ]:
# get the best tanimotos
spectrums_retrieved = [filt_spectrums_k_retrieved[index_spectrum][best_index]  for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]
tanimoto_retrieved = [filt_tanimoto_k_retrieved[index_spectrum][best_index]  for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]
max_sim = [filt_sim_k_retrieved[index_spectrum][best_index]  for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]
edit_distance=[filt_edit_distance_k_retrieved[index_spectrum][best_index]  for index_spectrum, best_index in enumerate(best_indexes) if best_index is not None]

In [ ]:
smiles_retrieved= [s.smiles for s in spectrums_retrieved]

In [ ]:
_=plt.hist(max_sim, color='r', bins=10)
plt.xlabel('maximum SIMBA predicted similarity found')
plt.ylabel('freq')
plt.grid()

In [ ]:
len(tanimoto_retrieved)

In [ ]:
plt.hist(tanimoto_retrieved,density=True, bins=10)
plt.grid()
plt.xlabel('tanimoto similarity with reference spectra')
plt.ylabel('frequency')
plt.ylim([0,3.5])

In [ ]:
tanimoto_retrieved=np.array(tanimoto_retrieved)
tanimoto_retrieved[tanimoto_retrieved<0.80].shape[0]/tanimoto_retrieved.shape[0]

In [ ]:
np.median(tanimoto_retrieved)

In [ ]:
plt.boxplot(tanimoto_retrieved)
plt.grid()

## Check which is the spectra that has wrong predictions

In [ ]:
target_index=-4

In [ ]:
diff_tanimoto =abs(np.array(tanimoto_retrieved)-np.array(max_sim))
bad_predictions=np.argsort(diff_tanimoto)[::-1]
#bad_predictions=np.argsort(abs(np.array(max_sim)))

In [ ]:
precursor_mass_query= [all_spectrums_janssen[bad_predictions[i]].params['pepmass'][0] for i in range(0,len(all_spectrums_janssen))]
precursor_mass_retrieved= [spectrums_retrieved[bad_predictions[i]].params['pepmass'][0] for i in range(0,len(all_spectrums_janssen))]

In [ ]:
diff_tanimoto_ordered= [diff_tanimoto[bad_predictions[i]] for i in range(0,len(all_spectrums_janssen))]

In [ ]:
plt.plot(diff_tanimoto_ordered, marker="*")
plt.xlabel('diff tanimoto')
plt.grid()

In [ ]:
plt.plot(precursor_mass_query)
plt.xlabel('precursor mz')
plt.grid()

In [ ]:
precursor_mass_ref= [s.params['pepmass'][0] for s in all_spectrums_reference]

In [ ]:
plt.hist(precursor_mass_ref, label='reference', bins=50, color='b',alpha=0.5, density=True)
plt.hist(precursor_mass_query, label='query', bins=50, color='r',alpha=0.5, density=True)
plt.grid()
plt.xlabel('precursor_mass')

In [ ]:
number_peaks_ref= np.array([len(s.mz) for s in all_spectrums_reference_processed])
number_peaks_query= np.array([len(s.mz) for s in all_spectrums_janssen_processed])
_=plt.hist(number_peaks_ref, bins=10, density=True, alpha=0.5,label='ref')
_=plt.hist(number_peaks_query, bins=10, density=True, alpha=0.5, label='janssen')
plt.ylabel('density')
plt.xlabel('num peaks')
plt.xlim([0,300])
plt.legend()
plt.grid()

In [ ]:
number_peaks_ref[number_peaks_ref<10].shape[0]/number_peaks_ref.shape[0]

In [ ]:
number_peaks_query[number_peaks_query<10].shape[0]/number_peaks_query.shape[0]

In [ ]:
target_spectra=bad_predictions[target_index]

In [ ]:
all_spectrums_janssen[target_spectra].params

In [ ]:
all_spectrums_janssen[target_spectra].mz

In [ ]:
all_spectrums_janssen[target_spectra].intensity/max(all_spectrums_janssen[target_spectra].intensity)

In [ ]:
spectrums_retrieved[target_spectra].params

In [ ]:
max(all_spectrums_janssen[0].intensity)

In [ ]:
max(spectrums_retrieved[6].intensity)

In [ ]:
len(spectrums_retrieved[target_spectra].mz)

In [ ]:
len(all_spectrums_janssen[target_spectra].mz)

In [ ]:
Plotting.plot_mirror_plot( all_spectrums_janssen[target_spectra],spectrums_retrieved[target_spectra])

In [ ]:
## pass through preprocessing:
pp=Preprocessor()
import copy

spectrum_copy= copy.deepcopy(spectrums_retrieved[target_spectra])
spectrum_copy=pp.preprocess_spectrum(spectrum_copy)

In [ ]:
len(spectrums_retrieved[target_spectra].intensity)

In [ ]:
len(max_sim)

In [ ]:
max_sim[target_spectra]

In [ ]:
tanimoto_retrieved[target_spectra]

In [ ]:
##plot edit distance
plt.title('edit distance probability')
plt.grid()
plt.bar(x=['>5','4','3','2','1','0'], height=edit_distance[target_spectra])

In [ ]:
edit_distance[target_spectra]

In [ ]:
sim_mces = MCES.calculate_mces_sim(smiles1=all_spectrums_janssen[target_spectra].params['smiles'], 
                         smiles2=spectrums_retrieved[target_spectra].params['smiles'])
sim_mces

In [ ]:
Chem.CanonSmiles(all_spectrums_janssen[target_spectra].params['smiles'])

In [ ]:
from rdkit import Chem
mol_janssen = Chem.MolFromSmiles(all_spectrums_janssen[target_spectra].params['smiles'])
mol_janssen

In [ ]:
Chem.CanonSmiles(spectrums_retrieved[target_spectra].params['smiles'])

In [ ]:
mol_ref = Chem.MolFromSmiles(spectrums_retrieved[target_spectra].params['smiles'])
mol_ref

In [ ]:
temp_smiles='C1=CC(=C(C=C1C2=CC(=O)C3=C(O2)C(=C(C=C3O)O)O)O)[O][C@@H]4[C@@H](O)[C@H](O[C@H]5[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@H]5O)[C@@H](O)[C@H](O[C@H]6[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@H]6O)[C@@H](O)[C@H](O[C@H]7[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@H]7O)[C@@H](O)[C@H](O[C@H]8[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@H]8O)O4'
Chem.MolFromSmiles(temp_smiles)

In [ ]:
Tanimoto.compute_tanimoto_from_smiles(all_spectrums_janssen[target_spectra].params['smiles'], temp_smiles)

## Explainability

In [ ]:
from src.transformers.sklearn_model import SklearnModel
sklearn_model = SklearnModel(model_loaded=ad_multitask.multitask_model)

In [ ]:
#explainer=sklearn_model.get_explainer(dataset['molecule_pairs_test'])

## Output of MS2Deepscore

In [ ]:
## BAD PREDICTIONS

results_simba_all=[]
results_tanimoto_all=[]

results_ms2_all=[]
for index in range(0,20):
    #get the index of the bad spectra
    t=bad_predictions[index]

    # get the tanimoto ground truth
    results_tanimoto_all.append(tanimoto_retrieved[t])

    # get the results of simba
    results_simba_all.append(max_sim[t])
    results_ms2_all.append(ms2_object.get_predictions_ms2deepscore(all_spectrums_janssen[t], spectrums_retrieved[t]))

In [ ]:
results_ms2_all

In [ ]:
plt.scatter(results_simba_all, results_ms2_all)
plt.ylim([0,1])
plt.xlim([0,1])
plt.grid()
plt.xlabel('SIMBA pred')
plt.ylabel('MS2Deepscore pred')

In [ ]:
plt.scatter(results_tanimoto_all, results_ms2_all, label='ms2deepscore')
plt.scatter(results_tanimoto_all, results_simba_all, label='SIMBA')
plt.legend()
plt.ylim([0,1])
plt.xlim([0,1])
plt.grid()
plt.xlabel('Tanimoto')
plt.ylabel('MS2Deepscore pred')

## FIND SIMILARITY BASED ON MCES

In [ ]:
smiles_janssen = [Chem.CanonSmiles(s.smiles) for s in all_spectrums_janssen]
smiles_reference = [Chem.CanonSmiles(s.smiles) for s in all_spectrums_reference]

In [ ]:
len(smiles_retrieved)

In [ ]:
len(smiles_janssen)

In [ ]:
mces_sims=[]
for s0,s1 in tqdm(zip(smiles_janssen, smiles_retrieved)):
    similarity= MCES.calculate_mces_sim(s0, s1)
    mces_sims.append(similarity)

In [ ]:
# Specify the bin width
bin_width = 0.1

# Calculate the number of bins based on the data range and bin width
bins = np.arange(0, 1 + bin_width, bin_width)

In [ ]:
plt.hist(tanimoto_retrieved,alpha=0.5, label='tanimoto sim.', density=True, bins=bins)
plt.hist([m for m in mces_sims if m is not None],alpha=0.5, label='mces sim.', density=True, bins=bins)
plt.xlabel('similarity')
plt.ylabel('density')
plt.legend()
plt.grid()

In [ ]:
plt.boxplot([m for m in mces_sims if m is not None])
plt.grid()

## Rankings

In [ ]:
format_file_unknown = "./notebooks/discovery_search/results/{}_results_analog_discovery_unknwon_compounds.pkl"

if USE_CASMI:
    file= format_file_unknown.format('tanimoto_similarity_casmi')
else:
    file= format_file_unknown.format('tanimoto_similarity')
with open(file, 'rb') as f:
        results_tani = dill.load(f)

In [ ]:
## load the tanimoto similarity data
## get the indexes in the reference data
smiles_best_candidate_loaded = [Chem.CanonSmiles(s) for s in results_tani['smiles_retrieved']]
smiles_reference_loaded = [Chem.CanonSmiles(s) for s in results_tani['smiles_reference']]
smiles_janssen_loaded = [Chem.CanonSmiles(s) for s in results_tani['smiles_janssen']]

In [ ]:
#list_rankings=[]

In [ ]:
#list_rankings=[]
list_rankings=AnalogDiscovery.get_rank_of_best_candidate(similarities, smiles_janssen, smiles_janssen_loaded, smiles_reference, smiles_best_candidate_loaded)

## saving of results

In [ ]:
len(max_sim), len(tanimoto_retrieved), len(all_spectrums_janssen), len(smiles_retrieved), len(mces_sims), len(smiles_janssen), len(spectrums_retrieved)

In [ ]:
results ={ 'preprocessed_all_spectrums_janssen':all_spectrums_janssen,
            'smiles_janssen':smiles_janssen,
          'smiles_retrieved':smiles_retrieved,
          'spectrums_retrieved':spectrums_retrieved,
            'tanimoto_retrieved':tanimoto_retrieved,
          'max_sim':max_sim,
          'mces_retrieved':mces_sims,
          'smiles_reference': smiles_reference,
          'list_rankings':list_rankings,
         'similarities': similarities,
          'preprocessed_all_spectrums_reference':all_spectrums_reference,
         }

In [ ]:
with open('./notebooks/discovery_search/results/'+similarity_name + '_results_analog_discovery_unknwon_compounds.pkl', 'wb') as f:
    dill.dump(results, f)

In [ ]:
similarity_name